In [21]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

In [22]:
engine = create_engine('sqlite:///college.db', echo=True)
meta = MetaData()

In [23]:
students = Table(
    'students', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('lastname', String),
)
meta.create_all(engine)

2022-04-03 18:53:45,614 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:53:45,615 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2022-04-03 18:53:45,615 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:53:45,617 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2022-04-03 18:53:45,617 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:53:45,619 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-03 18:53:45,619 INFO sqlalchemy.engine.Engine [no key 0.00074s] ()
2022-04-03 18:53:45,622 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
parcours = Table(
    'parcours', meta,
    Column('id_parcours', Integer, primary_key=True),
    Column('name', String),
    Column('student_id', Integer, ForeignKey("students.id")),
)
meta.create_all(engine)

2022-04-03 18:53:54,347 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:53:54,348 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2022-04-03 18:53:54,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:53:54,351 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parcours")
2022-04-03 18:53:54,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:53:54,352 INFO sqlalchemy.engine.Engine COMMIT


### ACID
- Atomicité : une transaction doit être effectuée en entier ou abandonnée, c'est à dire que si une opération est refusée, toutes les opérations de la transaction ne seront pas validées.
- Cohérente : l'état initial et final doivent être cohérents, c'est à dire respecter les contraintes d'intégrité.
- Isolation : Une transaction doit s'effectuer indépendamment des transactions qui s'exécutent en même temps, c'est à dire que les états intermédiaires d'une transaction sont cachés aux autres.
- Durabilité : lorsqu'une transaction est validée, ses effets ne peuvent être perdus.

In [25]:
values = [(1, "Jean", "Dubois"), (2, "Florent", "Desbiolles")]
# markers = ','.join('?' * len(values[0]))
# markers

In [26]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0]))
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=students.name, markers=markers)
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        except:
            transaction.rollback()
            raise
        transaction.commit()

2022-04-03 18:53:58,002 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:53:58,003 INFO sqlalchemy.engine.Engine INSERT OR REPLACE INTO students VALUES (?,?,?)
2022-04-03 18:53:58,004 INFO sqlalchemy.engine.Engine [raw sql] [(1, 'Jean', 'Dubois'), (2, 'Florent', 'Desbiolles')]
2022-04-03 18:53:58,006 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
values_parcours = [(1, "Data engineering", 1),(2, "Data Science", 2) ]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values_parcours[0]))
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'

            ins = ins.format(tablename=parcours.name, markers=markers)
            connection.execute(ins, values_parcours)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

2022-04-03 18:54:00,250 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:54:00,251 INFO sqlalchemy.engine.Engine INSERT OR REPLACE INTO parcours VALUES (?,?,?)
2022-04-03 18:54:00,252 INFO sqlalchemy.engine.Engine [raw sql] [(1, 'Data engineering', 1), (2, 'Data Science', 2)]
2022-04-03 18:54:00,253 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
with engine.connect() as connection:
    results = connection.execute("SELECT * FROM students;")
    print(results.fetchall())

with engine.connect() as connection:
    results = connection.execute("SELECT * FROM parcours;")
    print(results.fetchall())

2022-04-03 18:54:00,758 INFO sqlalchemy.engine.Engine SELECT * FROM students;
2022-04-03 18:54:00,759 INFO sqlalchemy.engine.Engine [raw sql] ()
[(1, 'Jean', 'Dubois'), (2, 'Florent', 'Desbiolles')]
2022-04-03 18:54:00,762 INFO sqlalchemy.engine.Engine SELECT * FROM parcours;
2022-04-03 18:54:00,762 INFO sqlalchemy.engine.Engine [raw sql] ()
[(1, 'Data engineering', 1), (2, 'Data Science', 2)]


In [9]:
# Exécutez la cellule pour afficher la réponse

sql = text('DROP TABLE IF EXISTS students;')
result = engine.execute(sql)

###Vérification que la table students n'existe plus en réutilisant le code d'insertion de tuple

values = [
   (1,"Jean", "Dubois"),
   (2,"Florent", "Desbiolles"),
   ]

with engine.connect() as connection:
   with connection.begin() as transaction:
       try:
           markers = ','.join('?' * len(values[0]))
           ins = 'INSERT INTO students VALUES ({markers})'
           ins = ins.format(tablename=students.name, markers=markers)
           connection.execute(ins, values)
       except:
           transaction.rollback()
           raise
       else:
           transaction.commit()

2022-04-03 18:51:53,063 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS students;
2022-04-03 18:51:53,064 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ()
2022-04-03 18:51:53,067 INFO sqlalchemy.engine.Engine COMMIT
2022-04-03 18:51:53,068 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:51:53,069 INFO sqlalchemy.engine.Engine INSERT INTO students VALUES (?,?,?)
2022-04-03 18:51:53,069 INFO sqlalchemy.engine.Engine [raw sql] [(1, 'Jean', 'Dubois'), (2, 'Florent', 'Desbiolles')]
2022-04-03 18:51:53,071 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: students
[SQL: INSERT INTO students VALUES (?,?,?)]
[parameters: [(1, 'Jean', 'Dubois'), (2, 'Florent', 'Desbiolles')]]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [29]:
inspector = inspect(engine)
inspector.get_table_names()

2022-04-03 18:54:05,316 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-04-03 18:54:05,317 INFO sqlalchemy.engine.Engine [raw sql] ()


['parcours', 'students']

In [30]:
inspector.get_columns(table_name='students')

2022-04-03 18:54:06,545 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("students")
2022-04-03 18:54:06,547 INFO sqlalchemy.engine.Engine [raw sql] ()


[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'name',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'lastname',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

In [31]:
inspector.get_foreign_keys(table_name='parcours')

2022-04-03 18:54:24,472 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("parcours")
2022-04-03 18:54:24,474 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:54:24,475 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-03 18:54:24,476 INFO sqlalchemy.engine.Engine [raw sql] ('parcours',)


[{'name': None,
  'constrained_columns': ['student_id'],
  'referred_schema': None,
  'referred_table': 'students',
  'referred_columns': ['id'],
  'options': {}}]

In [32]:
conn = engine.connect()

In [33]:
result = conn.execute('SELECT lastname FROM students;')
result.fetchall()

2022-04-03 18:55:30,308 INFO sqlalchemy.engine.Engine SELECT lastname FROM students;
2022-04-03 18:55:30,310 INFO sqlalchemy.engine.Engine [raw sql] ()


[('Dubois',), ('Desbiolles',)]

In [ ]:
stmt = text ( "SELECT  FROM ;")
result = conn.execute(stmt)
result.fetchall()

### HAVING
Cette clause fonctionne exactement comme la clause WHERE mais pour des fonctions d'agrégats. Elle permet de spécifier un prédicat sur le résultat d'un GROUP BY, elle est donc forcément précédée par cette dernière clause.
Si on utilise la clause WHERE, on élimine des lignes avant l'agrégation.

In [ ]:
stmt = text ( "SELECT CustomerId, SUM(Total) as tot FROM invoices GROUP BY CustomerId HAVING tot > 20 ;")
result = conn.execute(stmt)
result.fetchall()

In [ ]:
stmt = text ( "SELECT CustomerId, SUM(Total) AS tot FROM invoices WHERE Total > 20 GROUP BY CustomerId " )
result = conn.execute(stmt)
result.fetchall()

## Requetes imbriquées

La première requête récupère les IDs des 10 clients avec les plus gros montants.
La deuxième requête classe ces résultats par ordre croissant.

In [37]:
stmt = text("SELECT * FROM "
                "(SELECT CustomerId, SUM(Total) AS Tot "
                "FROM invoices GROUP BY CustomerId ORDER BY Tot DESC LIMIT 10) "
            "table_temp ORDER BY Tot")
result = conn.execute(stmt)
result.fetchall()

2022-03-23 15:04:34,378 INFO sqlalchemy.engine.Engine SELECT * FROM (SELECT CustomerId, SUM(Total) AS Tot FROM invoices GROUP BY CustomerId ORDER BY Tot DESC LIMIT 10) table_temp ORDER BY Tot
2022-03-23 15:04:34,379 INFO sqlalchemy.engine.Engine [generated in 0.00124s] ()
2022-03-23 15:04:34,380 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT * FROM (SELECT CustomerId, SUM(Total) AS Tot FROM invoices GROUP BY CustomerId ORDER BY Tot DESC LIMIT 10) table_temp ORDER BY Tot]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

### Jointures

Produit cartésien ou CROSS JOIN
Il s'agit de la requête de jointure la plus élémentaire.
Le résultat est ce que l'on appelle le produit cartésien. Chacune des lignes de la première table va être associée à chacune des lignes de la seconde table.

Jointure interne ou INNER JOIN
Ce type de jointure va permettre de concaténer les tuples des 2 tables deux à deux si une condition est satisfaite.
Cette condition peut être de tout type tant qu'elle retourne un booléen. Typiquement, cette condition sera l'égalité d'un attribut en commun ; une clé primaire pour l'une et une clé étrangère pour l'autre par exemple. La clé étrangère fait référence à la clé primaire de la première table.

Jointure à droite ou RIGHT JOIN
La jointure à droite ou RIGHT JOIN est une jointure entre 2 tables qui permet de retourner tous les enregistrements de la table de droite même s’il n’y a pas de correspondance avec la table de gauche. S'il n'y a pas de correspondance, les valeurs manquantes sont définies NULL.

stmt = text ( "SELECT * FROM artists INNER JOIN albums ON artists.ArtistId = albums.ArtistId LIMIT 10;" )
result = conn.execute(stmt)
result.fetchall()

### Modifier table

Modifier une ligne avec SET et UPDATE
On peut utiliser UPDATE et SET pour modifier une ou plusieurs ligne(s).
Par exemple, si on veut modifier la colonne colonne1 pour qu'elle prenne la valeur nouvelle_valeur lorsque la colonne colonne2 vaut valeur, on peut écrire :

UPDATE table1
SET colonne1 = nouvelle_valeur WHERE colonne2 = valeur;
Supprimer des lignes avec DELETE
Sur le même principe, on peut supprimer les lignes pour lesquelles la colonne colonne2 vaut valeur en faisant :

DELETE FROM table WHERE colonne2 = valeur;